Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import sys
from six.moves import cPickle as pickle

# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

First reload the data we generated in _notmist.ipynb_.

In [ ]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
datasets = {
    "image_size": 28,
    "label_count": 10
}
datasets["total_image_size"] = datasets["image_size"] * datasets["image_size"]

def reformat(dataset, labels, name):
    dataset = dataset.reshape((-1, datasets["total_image_size"])).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(datasets["label_count"]) == labels[:,None]).astype(np.float32)
    print(name + " set", dataset.shape, labels.shape)
    return dataset, labels
datasets["train"], datasets["train_labels"] = reformat(train_dataset, train_labels, "Training")
datasets["valid"], datasets["valid_labels"] = reformat(valid_dataset, valid_labels, "Validation")
datasets["test"], datasets["test_labels"] = reformat(test_dataset, test_labels, "Test")

print(datasets.keys())

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
def run_graph(graph_info, data, step_count):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        batch_size = graph_info["batch_size"]
        for step in xrange(step_count):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (data["train_labels"].shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = data["train"][offset:(offset + batch_size), :]
            batch_labels = data["train_labels"][offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            targets = [graph_info["optimizer"], graph_info["loss"], graph_info["train"]]
            feed_dict = {graph_info["tf_train"] : batch_data, graph_info["tf_train_labels"] : batch_labels}
            _, l, predictions = session.run(targets, feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(graph_info["valid"].eval(), data["valid_labels"]))
        print("Test accuracy: %.1f%%" % accuracy(graph_info["test"].eval(), data["test_labels"]))

### Logistic

In [ ]:
def setup_logistic(batch_size, rate_alpha, l2_beta, data):
    graph = tf.Graph()
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train = tf.placeholder(tf.float32, shape=(batch_size, data["total_image_size"]))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, data["label_count"]))
        tf_valid = tf.constant(data["valid"])
        tf_test = tf.constant(data["test"])

        # Variables.
        weights = tf.Variable(tf.truncated_normal([data["total_image_size"], data["label_count"]]))
        biases = tf.Variable(tf.zeros([data["label_count"]]))

        # Training computation.
        logits = tf.matmul(tf_train, weights) + biases
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        l2_loss = loss + l2_beta * tf.nn.l2_loss(weights)
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "tf_train": tf_train,
            "tf_train_labels": tf_train_labels,
            "loss": loss,
            # Optimizer.
            "optimizer": tf.train.GradientDescentOptimizer(rate_alpha).minimize(l2_loss),
            # Predictions for the training, validation, and test data.
            "train": tf.nn.softmax(logits),
            "valid": tf.nn.softmax(tf.matmul(tf_valid, weights) + biases),
            "test": tf.nn.softmax(tf.matmul(tf_test, weights) + biases)
        }

    return info

In [ ]:
logistic_graph = setup_logistic(batch_size=128, rate_alpha=0.5, l2_beta=0.01, data=datasets)

run_graph(logistic_graph, datasets, 3001)

### Neural Network

In [ ]:
def setup_neural(batch_size, hidden_size, rate_alpha, l2_beta, data):
    graph = tf.Graph()
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train = tf.placeholder(tf.float32, shape=(batch_size, data["total_image_size"]))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, data["label_count"]))
        tf_valid = tf.constant(data["valid"])
        tf_test = tf.constant(data["test"])

        # Hidden layer
        with tf.name_scope("hidden1"):
            hidden_weights = tf.Variable(tf.truncated_normal([data["total_image_size"], hidden_size]))
            hidden_biases = tf.Variable(tf.zeros([hidden_size]))
            train_hidden = tf.nn.relu(tf.matmul(tf_train, hidden_weights) + hidden_biases)
            valid_hidden = tf.nn.relu(tf.matmul(tf_valid, hidden_weights) + hidden_biases)
            test_hidden = tf.nn.relu(tf.matmul(tf_test, hidden_weights) + hidden_biases)

        with tf.name_scope("softmax_linear"):
            weights = tf.Variable(tf.truncated_normal([hidden_size, data["label_count"]]))
            biases = tf.Variable(tf.zeros([data["label_count"]]))
            train_logits = tf.matmul(train_hidden, weights) + biases
            valid_logits = tf.matmul(valid_hidden, weights) + biases
            test_logits = tf.matmul(test_hidden, weights) + biases

        # Training computation.
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels))
        l2_loss = loss + l2_beta * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "tf_train": tf_train,
            "tf_train_labels": tf_train_labels,
            "loss": loss,
            # Optimizer.
            "optimizer": tf.train.GradientDescentOptimizer(rate_alpha).minimize(l2_loss),
            # Predictions for the training, validation, and test data.
            "train": tf.nn.softmax(train_logits),
            "valid": tf.nn.softmax(valid_logits),
            "test": tf.nn.softmax(test_logits)
        }
    return info

In [ ]:
neural_graph = setup_neural(batch_size=128, hidden_size = 1024, rate_alpha=0.5, l2_beta=0.001, data=datasets)

run_graph(neural_graph, datasets, 3001)

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
dataslice = datasets.copy() # Shallow

slice_size = 4 * 128
dataslice["train"] = datasets["train"][:slice_size]
dataslice["train_labels"] = datasets["train_labels"][:slice_size]

print(dataslice["train"].shape)
print(dataslice["train_labels"].shape)

In [ ]:
logistic_graph_slice = setup_logistic(batch_size=128, rate_alpha=0.5, l2_beta=0.01, data=dataslice)

run_graph(logistic_graph_slice, dataslice, 3001)

In [ ]:
neural_graph_slice = setup_neural(batch_size=128, hidden_size = 1024, rate_alpha=0.5, l2_beta=0.001, data=dataslice)

run_graph(neural_graph_slice, dataslice, 3001)

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
